# CheatBot
##### The primary goal of this bot is to generate answers based on a piece of data given.

## TechStack
Gemini LLm <br>
Langchain <br>
GeminiEmbeddings <br>
ChromaDB as a Vector Database<br>
Various other python libraries to enhancing the bot<br>
<br>
#### - Developed by Mihiresh Joshi

In [1]:

# Install all these libraries using pip command
# Here I made your work easy (use 'pip3' if on pip3 version): 
# pip install PyPDF2 langchain python-dotenv pandas chromadb google-api-python-client tqdm

import PyPDF2
import langchain
import os
from dotenv import load_dotenv
from pprint import pprint
import pandas as pd
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
import google.generativeai as genai
from IPython.display import Markdown
from chromadb.api.types import Embeddings
import time
from tqdm import tqdm
from google.generativeai import GenerationConfig, GenerativeModel

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
api_key = "api key"  # Change it with your API key
genai.configure(api_key=api_key)

In [3]:
# A very basic function to extract text from pdf

def extract_text(pdf_path):
    extracted_text = ""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)
        for page_number in range(num_pages):
            page = pdf_reader.pages[page_number]
            extracted_text += page.extract_text()
    print(extracted_text)
    return extracted_text

pdf_path = " pdf address "  # Don't forget to give your address here 
text = extract_text(pdf_path)

Data Management & Visualization (DMV) Module 1Structured Query Language (SQL)Index
2Lecture  1Overview of SQL4Lecture  1Data Types, Data Definition Commands. 16Lecture  2Data Manipulation commands32Lecture  2Integrity constraints ; Referential integrity , check constraints 38Lecture 1 Overview of SQLOverview of SQL- What is SQL?§Standard command set used to communicate with relational DBMS.§ All tasks related to Relational Database Management like creating table, querying the database for information , modifying data in the database.Advantages§High level  language that provides a greater degree of abstraction than procedural languages.§Allow the end-users to deal with number of database management systems where it is available.§Simple and easy to learn can handle complex situation.§All SQL operations are performed at set level.§For digital systems, the variable takes on discrete values.4Data Models
5Model ofsystemin client’smindEntity model ofclient’s modelTable modelof entity modelTab

In [4]:
# text = text.replace("Module 1– Introduction to Social Network"," ")
# text

In [5]:
# The most important step of chunking your textual data. Here you could use complete creativity while fine-tuning
# My suggestion try performing content-based chunking

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,    #for next time let's try to keep the chunk size little bigger
    chunk_overlap = 100,
    length_function = len,
    add_start_index = True
)

In [6]:
texts = text_splitter.create_documents([text])
print(texts[14].page_content)

50create table course (    course_id char(5) primary key,    title             varchar(20), dept_name varchar(20) references department)create table course (    …    dept_name varchar(20),    foreign key (dept_name) references department                on delete cascade                on update cascade,    . . . )alternative actions to cascade:  set null, set defaultThe CHECK Constraint§Defines a condition that each row must satisfy..., salary NUMBER(2)     CONSTRAINT emp_salary_min              CHECK (salary > 0),...Adding a Constraint Syntax§Use the ALTER TABLE statement to:•Add or drop a constraint, but not modify its structure•Enable or disable constraints•Add a NOT NULL constraint by using the MODIFY clauseALTER TABLE  table  ADD [CONSTRAINT constraint] type (column);Adding a ConstraintAdd a FOREIGN KEY constraint to the EMPLOYEES table indicating that a manager must already exist as a valid employee in the EMPLOYEES table.ALTER TABLE     employeesADD CONSTRAINT  emp_manager_fk


In [7]:
# Combinig chunks of your textual data to form a List. 

docs = []

for chunk in texts:
    docs.append(chunk.page_content)

print(docs[:2])

['Data Management & Visualization (DMV) Module 1Structured Query Language (SQL)Index\n2Lecture  1Overview of SQL4Lecture  1Data Types, Data Definition Commands. 16Lecture  2Data Manipulation commands32Lecture  2Integrity constraints ; Referential integrity , check constraints 38Lecture 1 Overview of SQLOverview of SQL- What is SQL?§Standard command set used to communicate with relational DBMS.§ All tasks related to Relational Database Management like creating table, querying the database for information , modifying data in the database.Advantages§High level  language that provides a greater degree of abstraction than procedural languages.§Allow the end-users to deal with number of database management systems where it is available.§Simple and easy to learn can handle complex situation.§All SQL operations are performed at set level.§For digital systems, the variable takes on discrete values.4Data Models', '5Model ofsystemin client’smindEntity model ofclient’s modelTable modelof entity mo

In [8]:
class GeminiEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        model = 'models/embedding-001'
        title = 'LinkedIn API'
        return genai.embed_content(
            model=model,
            content=input,
            task_type="retrieval_document",
            title=title)["embedding"]
        

In [9]:
# Function to define and create you Vector Databse of chromadb in Database folders

def create_chroma_db(docs,name):
    chroma_client = chromadb.PersistentClient(path="db address") #Don't forget to change path
    db = chroma_client.get_or_create_collection(
        name=name, embedding_function=GeminiEmbeddingFunction())
    
    initial_size = db.count()
    for i, d in tqdm(enumerate(docs), total=len(docs), desc="Creating Chroma DB"):
        db.add(
            documents=d,
            ids=str(i + initial_size)
        )
        time.sleep(0.5)
    return db


def get_chroma_db(name):
    chroma_client = chromadb.PersistentClient(path="db address 2") # Here as well 
    return chroma_client.get_collection(name=name, function=EmbeddingFunction())

In [10]:
# Creating your Vector Database. [Don't unneccessarily run this code]

db = create_chroma_db(docs, "sme_db")
db.count()

Creating Chroma DB: 100%|██████████| 131/131 [02:15<00:00,  1.04s/it]


11228

In [11]:
pd.DataFrame(db.peek(11097))

,ids,embeddings,metadatas,documents,uris,data
0,0,"[0.015172463841736317, -0.010413325391709805, ...",None,Python Tutorial\nRelease 3.7.0\nGuido van Ross...,None,None
1,1,"[0.03553083539009094, 0.01233371626585722, -0....",None,4.3 The range() Function . . . . . . . . . . ....,None,None
2,10,"[0.019464796409010887, -0.03204401955008507, -...",None,"and tools, and additional documentation.\nThe ...",None,None
3,100,"[-0.0029656183905899525, 0.003489450318738818,...",None,Yuck!\nC>\nThese two variables are only deﬁned...,None,None
4,1000,"[-0.03242751583456993, -0.009994188323616982, ...",None,"compare unequal (except with themselves), and ...",None,None
...,...,...,...,...,...,...
11092,9877,"[-0.009271173737943172, -0.08612344413995743, ...",None,UNDER_DELIVER Y_AUDIENCE_EXP ANSION\nOVER_DELI...,None,None
11093,9878,"[0.02701648883521557, -0.07625408470630646, -0...",None,urn:li:sponsoredCampaign:12244313 and urn:li:s...,None,None
11094,9879,"[0.03511833772063255, -0.06084388121962547, -0...",None,HTTP\nField\nNameRequir ed Descr iption\nentit...,None,None
11095,988,"[0.012748446315526962, 0.019533473998308182, -...",None,Zero-dimensional buﬀers are C and Fortran cont...,None,None


In [12]:
len(pd.DataFrame(db.peek(10)).iloc[0]["embeddings"])

768

In [13]:
# Function to retrieve fact from the vector database

def get_relevant_passages(query, db, n_results):
    passages = db.query(query_texts=[query], n_results=n_results)['documents'][0]
    return passages

In [14]:
model = genai.GenerativeModel('gemini-pro')

In [15]:
chats = {}

In [16]:
def extract_text_from_response(response):
    # Initialize an empty string to accumulate text
    extracted_text = ""
    
    # Check if the response has a 'parts' attribute and iterate over it if present
    if hasattr(response, 'parts'):
        for part in response.parts:
            if hasattr(part, 'text'):  # Ensure the part has a 'text' attribute
                extracted_text += part.text + "\n"  # Append the text from each part
    
    # Alternatively, if the structure is deeper (e.g., candidates with content parts):
    elif hasattr(response, 'candidates') and len(response.candidates) > 0:
        for candidate in response.candidates:
            for part in candidate.content.parts:
                if hasattr(part, 'text'):
                    extracted_text += part.text + "\n"
    
    return extracted_text.strip()  # Return the combined text, stripping any trailing newline


In [106]:
def make_prompt(ques, knowledge, chats):
    text = knowledge.replace("'","").replace('"','') #even i dont know why i did this


    # Choose from the given prompts which suits you the BEST
    # If you want you may create your own prompt by referring to the following options
    # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


    # prompt = f"""question: {ques}.\n
    # information base or knowledge base: {text}\n
    # Answer the question in 1250-2500 words strictly based on the information base or knowledge base given, \n
    # If the question is asking for a code then also explain the algorithm, if the question is asking for career guidance then provide complete career guidance and links for various courses, if difference is asked differentiate with minimum 7 points in tabular format and one example
    # if the information is not sufficient then give output as "Info not good to answer"
    # """

    # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

    # prompt = f"""question: {ques}.\n
    # information base or knowledge base: {text}\n
    # Answer the question strictly based from {text} by filtering the required information from {text}\n
    # Generate a sophisticated and neat answer such that it could be written in exam\n
    # if the information is not sufficient then use keywords from {text} related to {ques} and answer the queestion and at the end of answer mention "[from other sources as well]" BUT try to give answers from only and only{text}\n
    # If the question is asking for a code then also explain the algorithm,\n 
    # if the question is asking for career guidance then provide complete career guidance and links for various courses, \n
    # if difference is asked differentiate with minimum 7 points in tabular format and one example
    # MOST IMPORTANTLY: answer the questions from {text} in such a way to write in exam\n
    # NOTE: try to answwer {ques} from {text} as much as possible
    # """

    # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

    # prompt = f"""question: {ques}.\n
    # chat histoty: {chats}\n
    # information base or knowledge base: {text}\n
    # Work like a answer generarting bot for exam who generates answer for given {ques} based on the {text} provided. Generate output in such a way that it should be written in examination.
    # check whether {ques} is a question or an instruction, if its an instruction give an affirmative response \n
    # ELSE
    # Answer the question STRICTLY based from {text} by filtering the required information from {text}\n
    # if the information is not sufficient then use keywords from {text} related to {ques} and answer the queestion and at the end of answer mention "[from other sources as well]" BUT try to give answers from only and only{text}\n
    # If the question is asking for a code then also explain the algorithm,\n 
    # if the question is asking for career guidance then provide complete career guidance and links for various courses, \n
    # if difference is asked differentiate with minimum 7 points in tabular format and one example\n
    # Also DO NOT forget to consider the previous qquestions and the answers you gave for it :{chats}\n
    # If the question refers about past questions or answers, generate the answer considering {chats} and {text}
    # MOST IMPORTANTLY: answer the questions from {text} in such a way to write in exam\n
    # NOTE: try to answwer {ques} from {text} as much as possible, and bigger answer is prefered
    # """

    # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

    prompt = f"""Given the question '{ques}', the previous chat history '{chats}', and a knowledge base '{text}':
    1. Identify if '{ques}' is a question or an instruction. For instructions, provide an affirmative response.
    2. If it's a question, answer it based on the information in '{text}'. Generate the answer which is suitable for an exam setting.
    3. If '{ques}' requires information beyond '{text}', use related keywords from '{text}' to answer but don't left any question unanswered, and mention [from other sources as well] at the end.
    4. For coding questions, include an algorithm explanation. For career guidance inquiries, offer comprehensive advice and resource links. If asked to differentiate, provide at least 7 comparison points in a table with an example.
    5. Incorporate relevant details from the chat history '{chats}' when answering, especially if the question refers to previous discussions.
    6. Aim to source answers primarily from '{text}', ensuring the response is detailed (answer should be big) and exam-appropriate.
    """ 

    gen_config = GenerationConfig(temperature=0.7)
    answer_text = model.generate_content(prompt,generation_config=gen_config)
    answer = extract_text_from_response(answer_text)
    
    return answer


# <span style="color:yellow">Question:-</span>

In [135]:
ques = " Draw the star schema for previous answer "
passages = get_relevant_passages(ques, db, n_results=25) #i have kept the n_results more because i wanted more info to be included in my answwer
txt = ""
for passage in passages:
    txt += passage

In [136]:
def list_to_string(passages):
    content = ""
    for passage in passages:
        content += passage + "\n"
    return content

cont = list_to_string(passages)


In [137]:
cont = cont.replace('Module :1 Introduction to Big Data and Hadoop',' ') #this is subjective for this data
cont = cont.replace('Lecture 1: Introduction to Big Data3Lecture 2: Hadoop Ecosystem 58  Lecture 3: Introduction to spark3Lecture 4: Hadoop vs Spark58',' ') #this is subjective for this data
cont = cont.replace('\n',' ')  #this is subjective for this data
cont

'a valid employee in the EMPLOYEES table.ALTER TABLE     employeesADD CONSTRAINT  emp_manager_fk   FOREIGN KEY(manager_id)   REFERENCES employees(employee_id); a valid employee in the EMPLOYEES table.ALTER TABLE     employeesADD CONSTRAINT  emp_manager_fk   FOREIGN KEY(manager_id)   REFERENCES employees(employee_id); a valid employee in the EMPLOYEES table.ALTER TABLE     employeesADD CONSTRAINT  emp_manager_fk   FOREIGN KEY(manager_id)   REFERENCES employees(employee_id); "lifecycleState" : "PUBLISHED" ,            "lastModifiedAt" : 1648515427782 ,            "visibility" : "PUBLIC" ,            "publishedAt" : 1648515427701 ,            "author" : "urn:li:organization:1234567" ,            "distribution" : {                "feedDistribution" : "NONE",                "thirdPartyDistributionChannels" : []            },            "content" : {                "carousel" : {                    "cards": [                        {                            "landingPage" : "http://www.lin

In [138]:
answer = make_prompt(ques, cont, chats)

In [139]:
answer

'**Exam-Appropriate Response:**\n\n**Question:** Draw the star schema for the previous answer.\n\n**Answer:**\n\nA star schema is a data warehouse design pattern that is optimized for querying data from multiple dimensions. It consists of a fact table and multiple dimension tables. The fact table contains the main business data, while the dimension tables contain the descriptive attributes for each dimension.\n\nIn the previous answer, the fact table is the `Sales` table, which contains the following columns:\n\n* `Product_ID`\n* `Order_ID`\n* `Customer_ID`\n* `Employee_ID`\n* `Total_Sales`\n* `Quantity_Sold`\n* `Discount`\n\nThe dimension tables are:\n\n* `Product` table, which contains the following columns:\n    * `Product_ID`\n    * `Product_Name`\n    * `Product_Category`\n    * `Unit_Price`\n* `Customer` table, which contains the following columns:\n    * `Customer_ID`\n    * `Customer_Name`\n    * `Customer_Address`\n    * `Customer_City`\n    * `Customer_State`\n    * `Customer

In [140]:
# Function to add a new Q&A pair
def chat_history(ques, answer, chats):
    next_id = len(chats) + 1
    chats[next_id] = {"question": ques, "answer": answer}
if len(chats):
    if (chats[len(chats)]['question']!=ques or chats[len(chats)]['question']!=ques):
        chat_history(ques=ques, answer=answer, chats=chats)
    else:
        print("same")
else:
    chat_history(ques=ques, answer=answer, chats=chats)

# <span style="color:yellow">Answer</span>

In [141]:
Markdown(answer)

**Exam-Appropriate Response:**

**Question:** Draw the star schema for the previous answer.

**Answer:**

A star schema is a data warehouse design pattern that is optimized for querying data from multiple dimensions. It consists of a fact table and multiple dimension tables. The fact table contains the main business data, while the dimension tables contain the descriptive attributes for each dimension.

In the previous answer, the fact table is the `Sales` table, which contains the following columns:

* `Product_ID`
* `Order_ID`
* `Customer_ID`
* `Employee_ID`
* `Total_Sales`
* `Quantity_Sold`
* `Discount`

The dimension tables are:

* `Product` table, which contains the following columns:
    * `Product_ID`
    * `Product_Name`
    * `Product_Category`
    * `Unit_Price`
* `Customer` table, which contains the following columns:
    * `Customer_ID`
    * `Customer_Name`
    * `Customer_Address`
    * `Customer_City`
    * `Customer_State`
    * `Customer_Zip_Code`
* `Employee` table, which contains the following columns:
    * `Employee_ID`
    * `Employee_Name`
    * `Employee_Title`
    * `Employee_Department`
    * `Employee_Region`
* `Time` table, which contains the following columns:
    * `Order_ID`
    * `Order_Date`
    * `Order_Year`
    * `Order_Quarter`
    * `Order_Month`

The relationships between the tables are as follows:

* The `Sales` table is related to the `Product` table by the `Product_ID` column.
* The `Sales` table is related to the `Customer` table by the `Customer_ID` column.
* The `Sales` table is related to the `Employee` table by the `Employee_ID` column.
* The `Sales` table is related to the `Time` table by the `Order_ID` column.

The star schema for the previous answer is shown below:

```
Sales
| Product_ID | Order_ID | Customer_ID | Employee_ID | Total_Sales | Quantity_Sold | Discount |
|---|---|---|---|---|---|---|
| 1 | 1 | 1 | 1 | 100.00 | 10 | 0.10 |
| 2 | 2 | 2 | 2 | 200.00 | 20 | 0.15 |
| 3 | 3 | 3 | 3 | 300.00 | 30 | 0.20 |
| 4 | 4 | 4 | 4 | 400.00 | 40 | 0.25 |
| 5 | 5 | 5 | 5 | 500.00 | 50 | 0.30 |

Product
| Product_ID | Product_Name | Product_Category | Unit_Price |
|---|---|---|---|
| 1 | Product A | Category A | 10.00 |
| 2 | Product B | Category B | 20.00 |
| 3 | Product C | Category C | 30.00 |
| 4 | Product D | Category D | 40.00 |
| 5 | Product E | Category E | 50.00 |

Customer
| Customer_ID | Customer_Name | Customer_Address | Customer_City | Customer_State | Customer_Zip_Code |
|---|---|---|---|---|---|
| 1 | Customer A | 123 Main Street | Anytown | CA | 12345 |
| 2 | Customer B | 456 Elm Street | Hometown | NY | 54321 |
| 3 | Customer C | 789 Oak Street | Cityville | TX | 67890 |
| 4 | Customer D | 1011 Pine Street | Woodville | FL | 23456 |
| 5 | Customer E | 1213 Maple Street | Springville | IL | 34567 |

Employee
| Employee_ID | Employee_Name | Employee_Title | Employee_Department | Employee_Region |
|---|---|---|---|---|
| 1 | Employee A | Salesperson | Sales | East |
| 2 | Employee B | Manager | Management | West |
| 3 | Employee C | Analyst | Analytics | Central |
| 4 | Employee D | Developer | Development | South |
| 5 | Employee E | Tester | Testing | North |

Time
| Order_ID | Order_Date | Order_Year | Order_Quarter | Order_Month |
|---|---|---|---|---|
| 1 | 2023-01-01 | 2023 | 1 | 1 |
| 2 | 2023-02-01 | 2023 | 1 | 2 |
| 3 | 2023-03-01 | 2023 | 1 | 3 |
| 4 | 2023-04-01 | 2023 | 2 | 1 |
| 5 | 2023-05-01 | 2023 | 2 | 2 |
```

This star schema can be used to answer queries such as:

* What is the total sales for each product?
* What is the average sales price for each product category?
* Who are the top 10 customers in terms of total sales?
* What is the sales trend over time?

In [142]:
if len(chats)>1:
    print(chats[len(chats)-1])
print(chats[len(chats)])
print(f"Length of chats is: {len(chats)}")

{'question': ' Write a set of SQL queries to implement Star Schema ', 'answer': "**SQL Queries to Implement Star Schema:**\n\n```sql\n-- Create the fact table\nCREATE TABLE FactSales (\n  Product_ID INT NOT NULL,\n  Order_ID INT NOT NULL,\n  Customer_ID INT NOT NULL,\n  Employee_ID INT NOT NULL,\n  Total_Sales DECIMAL(10, 2) NOT NULL,\n  Quantity_Sold INT NOT NULL,\n  Discount DECIMAL(5, 2) NOT NULL,\n  PRIMARY KEY (Product_ID, Order_ID)\n);\n\n-- Create the dimension tables\nCREATE TABLE DimProduct (\n  Product_ID INT NOT NULL,\n  Product_Name VARCHAR(50) NOT NULL,\n  Product_Category VARCHAR(50) NOT NULL,\n  Unit_Price DECIMAL(10, 2) NOT NULL,\n  PRIMARY KEY (Product_ID)\n);\n\nCREATE TABLE DimCustomer (\n  Customer_ID INT NOT NULL,\n  Customer_Name VARCHAR(50) NOT NULL,\n  Customer_Address VARCHAR(100) NOT NULL,\n  Customer_City VARCHAR(50) NOT NULL,\n  Customer_State VARCHAR(2) NOT NULL,\n  Customer_Zip_Code VARCHAR(10) NOT NULL,\n  PRIMARY KEY (Customer_ID)\n);\n\nCREATE TABLE Dim

In [143]:
Markdown(chats[1]['answer'])
# n = len(chats)
# id = 1
# while (id<=n):
#     Markdown(chats[id]['question'])
#     Markdown(chats[id]['answer'])
#     id += 1

**Differentiate between Data Node and Name Node**

| Feature | Data Node | Name Node |
|---|---|---|
| Purpose | Stores the actual data | Manages the file system namespace and regulates access to data |
| Location | On every machine in the Hadoop cluster | On the master node |
| Number of instances | Multiple | Single |
| Data storage | Data is stored in blocks on the local file system | Maintains a metadata table that tracks the location of data blocks |
| Data access | Applications read and write data directly from and to Data Nodes | Applications interact with the Name Node to determine the location of data blocks |
| Fault tolerance | Data is replicated across multiple Data Nodes | Maintains a backup of the metadata table on a secondary Name Node |
| Scalability | Can be easily scaled by adding more Data Nodes to the cluster | More difficult to scale, as it is a single point of failure |
| Performance | Can be optimized for specific workloads by tuning the local file system | Can be optimized for specific workloads by tuning the metadata management system |

**Example:**

In a Hadoop cluster, the Name Node stores the metadata for the file "/user/hadoop/input.txt". This metadata includes the location of the data blocks that make up the file. When an application wants to read the file, it first contacts the Name Node to get the location of the data blocks. The application then reads the data directly from the Data Nodes.

In [144]:
chats

{1: {'question': ' Differentiate between Data Node and Name Node ',
  'answer': '**Differentiate between Data Node and Name Node**\n\n| Feature | Data Node | Name Node |\n|---|---|---|\n| Purpose | Stores the actual data | Manages the file system namespace and regulates access to data |\n| Location | On every machine in the Hadoop cluster | On the master node |\n| Number of instances | Multiple | Single |\n| Data storage | Data is stored in blocks on the local file system | Maintains a metadata table that tracks the location of data blocks |\n| Data access | Applications read and write data directly from and to Data Nodes | Applications interact with the Name Node to determine the location of data blocks |\n| Fault tolerance | Data is replicated across multiple Data Nodes | Maintains a backup of the metadata table on a secondary Name Node |\n| Scalability | Can be easily scaled by adding more Data Nodes to the cluster | More difficult to scale, as it is a single point of failure |\n| P

# <span style="color:red">Danger</span>

In [145]:
# This will erase your chat history after a limit. Created this part because sometimes my Gemini LLM generates an error-500: Internal server error, it causes due to overloading of the model.
# chats = {}

#### /Users/mihiresh/Mihiresh/Hackathon/gdsc2/api_testing/rag2.ipynb